In [ ]:
# internal imports:
import utils.datareader as datareader
import utils.visualizer as visualizer
from models.transformer import *

# external imports:
import numpy as np
import pandas as pd

In [ ]:
datafolder = './data' # folder for where the data sets are
modelsavename = 'trafo'
predictionsavename = 'prediction_output.csv'

targets = [f'Bolt_{n+1}_Tensile' for n in range(6)]
learning_rate = 3e-4 # optimizer
learning_rate_annealing = 3e-5 # how much the learning rate is annealed every episode
episodes = 50000 # how often it is iterated over the entire dataset
inputlength = 60*4 # input sequence length
batchsize = 50 # batch size
save_model = 500 # episodes when the model is saved

# transformer model hyperparameters:
num_layers=6
dropout=0.05
no_heads=8
hidden_size=no_heads*2*6


Read and scale datasets:

In [ ]:
df, dfT = datareader.df_from_folder(datafolder)
df, dfT, scaler = datareader.normalize_input(df, dfT)

features = list(dfT.columns) # find features

Remove overhead data, find the nan timestamps and mask nans:

In [ ]:
# remove overhead:
df = df[targets+features] 
# find nan timestamps:
nans = np.array(df[targets].isna().product(1)) * np.arange(df.shape[0]) 
nans = nans[nans != 0]
# mask nans:
df = df.fillna(0)
dfT = dfT.fillna(0)

Set up model:

In [ ]:
try:
    net = torch.load(datafolder+'//'+modelsavename)
except Exception:
    net = Network(df.shape[1], len(targets), hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, no_heads=no_heads, maxlen=inputlength)
net.train() # starting in training mode
optim = torch.optim.Rprop(net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

Optimize parameters:

In [ ]:
plot_episode = 10 # which episodes to plot
losshistory = []
for e in range(episodes):
    # zero gradients out:
    net.zero_grad() 
    # sample a batch:
    x, y = sample(df, nans, targets, inputlength, batchsize, device=net.device)
    # predict:
    y_pred = net(x)      
    # calculate losses:
    loss = loss_fn(y_pred, y)
    # backpropagation:
    loss.backward()
    optim.step()       
    # save losses:
    losshistory.append(float(loss.detach().cpu().numpy()))
    # visualize losses:
    if e % plot_episode == 0:
        visualizer.lossplot(e+1, losshistory)
    
    # anneal learning rate:
    for g in optim.param_groups:
        g['lr'] = g['lr'] * (1 - learning_rate_annealing)
    # save model:
    if e % save_model == 0:
        torch.save(net, datafolder+'//'+modelsavename)



In [ ]:
net.eval()

x = torch.tensor(df.iloc[-inputlength:].to_numpy(), dtype=torch.float).to(net.device).unsqueeze(0)
predictions = []
for i in range(dfT.shape[0]):
    # predict:
    y_pred = net(x).detach()
    # add to prediction container:
    predictions.append(y_pred.cpu().numpy()[0,0])
    # add predictions to features:
    x_pred = torch.tensor(dfT.iloc[i], dtype=torch.float).to(net.device).unsqueeze(0).unsqueeze(0)
    x = torch.cat(
        (
            x, torch.cat((x_pred, y_pred),-1)
        ), 1
    )[:,-inputlength:,:]
    # print update:
    print(f'generated {i+1}|{dfT.shape[0]}')
predictions = np.array(predictions)


Generate output:

In [ ]:
predictionoutput = pd.DataFrame(predictions, columns=targets, index=dfT.index) # make dataframe
predictionoutput = datareader.rescale_output(predictionoutput, scaler) # rescale outputs
predictionoutput = predictionoutput[~np.array(dfT.isna().product(1), dtype=bool)] # remove the missing values again
predictionoutput.to_csv(datafolder+'//'+predictionsavename)